## Mode of Simulation 

Three type of simulation 

<strong>1. Profit maximization</strong> 

<strong>2. Solo Rides</strong>

<strong>3. Pooled Rides</strong>

## Load ExMAS and MaaSSim

In [36]:
%load_ext autoreload
%autoreload 2
import os, sys # add MaaSSim to path (not needed if MaaSSim is already in path)
module_path = os.path.abspath(os.path.join('../..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
from MaaSSim.utils import get_config, load_G, prep_supply_and_demand, generate_demand, generate_vehicles, initialize_df  # simulator
from MaaSSim.data_structures import structures as inData
from MaaSSim.simulators import simulate
from MaaSSim.visualizations import plot_veh
from MaaSSim.shared import prep_shared_rides
#from MaaSSim.performance import kpi_pax_pricing
#from MaaSSim.performance import kpi_veh_pricing


import logging
import matplotlib.pyplot as plt


import pandas as pd
import ExMAS

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Delft, Netherlands

In [37]:
params = get_config('../../data/config/delft.json')  # load configuration

params.times.pickup_patience = 180# 1 hour of simulation
params.times.patience = 180
params.simTime = 4 # 6 minutes hour of simulation
params.nP = 500 # reuqests (and passengers)
params.nV = 10 # vehicles


## Parameters for ExMAS

In [38]:
params.t0 = pd.Timestamp.now()
params.shareability.avg_speed = params.speeds.ride
params.shareability.shared_discount = 0.25
params.shareability.delay_value = 1
params.shareability.WtS = 1.3
params.shareability.price = 1.5 #eur/km
params.shareability.VoT = 0.0035 #eur/s
params.shareability.matching_obj = 'u_veh' #minimize VHT for vehicles
params.shareability.pax_delay = 0
params.shareability.horizon = 600
params.shareability.max_degree = 4
params.shareability.nP = params.nP
params.shareability.share = 1
params.shareability.without_matching = True
params.shareability.operating_cost = 0.0005
params.shareability.comm_rate = 0.2 

#params.platforms.comm_rate = 0.2 
#params.shareability.comm_rate = float(0.30)

inData = load_G(inData, params)  # load network graph 

inData = generate_demand(inData, params, avg_speed = False)
inData.vehicles = generate_vehicles(inData,params.nV)
inData.vehicles.platform = inData.vehicles.apply(lambda x: 0, axis = 1)
inData.passengers.platforms = inData.passengers.apply(lambda x: [0], axis = 1)
inData.requests['platform'] = inData.requests.apply(lambda row: inData.passengers.loc[row.name].platforms[0], axis = 1) 
inData.platforms = initialize_df(inData.platforms)
inData.platforms.loc[0]=[1,'Uber',30]
params.shareability.share = 1
params.shareability.without_matching = True


# params.kpi = 1 (Profit Maximazation)
# params.kpi = 2 (Solo Rides)
# params.kpi = 3 (Pooled Rides)

In [39]:
params.kpi = 2

In [40]:
#inData = ExMAS.main(inData, params.shareability) # create shareability graph (ExMAS) 
inData = ExMAS.main(inData, params.shareability, plot = False)

13-09-23 13:00:06-INFO-Initializing pairwise trip shareability between 200 and 200 trips.
13-09-23 13:00:06-INFO-creating combinations
13-09-23 13:00:06-INFO-39800	 nR*(nR-1)
13-09-23 13:00:07-INFO-Reduction of feasible pairs by 99.85%
13-09-23 13:00:07-INFO-Degree 2 	Completed
13-09-23 13:00:07-INFO-trips to extend at degree 2 : 181
13-09-23 13:00:07-INFO-At degree 2 feasible extensions found out of 17 searched
13-09-23 13:00:07-INFO-Degree 3 	Completed
13-09-23 13:00:07-INFO-trips to extend at degree 3 : 17
13-09-23 13:00:07-INFO-At degree 3 feasible extensions found out of 1 searched
13-09-23 13:00:07-INFO-Degree 4 	Completed
13-09-23 13:00:07-INFO-Max degree reached 4
13-09-23 13:00:07-INFO-Trips still possible to extend at degree 4 : 1


In [41]:
inData = prep_shared_rides(inData, params.shareability) # prepare schedules

13-09-23 13:00:07-INFO-Matching 200 trips to 399 rides in order to min u_veh
13-09-23 13:00:07-INFO-Problem solution: Optimal. 
Total costs for single trips:         52,303 
reduced by matching to:               46,846


In [42]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],6.687500,361,1,[6.6875],"[0, 361]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44811672, 630748802]","[None, 0, 0]",node time req_id od 0 N...,361,3.61,5.424000,1.084800,4.3392
1,[1],2.876500,155,1,[2.8765],"[112, 155]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44871798, 44884119]","[None, 1, 1]",node time req_id od 0 NaN...,155,1.55,2.334000,0.466800,1.8672
2,[2],7.346000,397,1,[7.346],"[122, 397]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1448535781, 44844068]","[None, 2, 2]",node time req_id od 0 ...,397,3.97,5.956500,1.191300,4.7652
3,[3],1.154500,62,1,[1.1545],"[206, 62]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44844874, 3510592637]","[None, 3, 3]",node time req_id od 0 ...,62,0.62,0.937500,0.187500,0.7500
4,[4],5.656000,305,1,[5.656],"[340, 305]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44867170, 1391531584]","[None, 4, 4]",node time req_id od 0 ...,305,3.05,4.588500,0.917700,3.6708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,"[16, 14, 24]",23.472025,1129,30,"[10.844825, 6.5196000000000005, 6.107600000000...","[678.5, 334, 374, 92, 26, 303]","[16, 14, 24]","[14, 16, 24]",3,394,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1402613351, 44840852, 3581094285, 44795...","[None, 16, 14, 24, 14, 16, 24]",node time req_id od 0 ...,1129,11.29,15.279750,3.055950,12.2238
395,"[48, 46, 49]",17.805225,664,32,"[8.90535, 4.378725, 4.5211500000000004]","[2964.0, 15, 207, 75, 209, 158]","[48, 46, 49]","[46, 49, 48]",3,395,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1410536956, 44833192, 44858073, 4487134...","[None, 48, 46, 49, 46, 49, 48]",node time req_id od 0 ...,664,6.64,12.085875,2.417175,9.6687
396,"[48, 46, 44]",17.678400,635,32,"[8.932649999999999, 4.560725, 4.185025]","[2964.0, 15, 142, 175, 78, 225]","[48, 46, 44]","[46, 44, 48]",3,396,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1410536956, 44833192, 44864086, 4487134...","[None, 48, 46, 44, 46, 44, 48]",node time req_id od 0 ...,635,6.35,11.727000,2.345400,9.3816
397,"[48, 49, 55]",17.133850,729,32,"[9.499125, 4.992075, 2.6426499999999997]","[2877.5, 222, 307, 22, 121, 57]","[48, 49, 55]","[49, 55, 48]",3,397,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1410536956, 44858073, 4655063484, 23419...","[None, 48, 49, 55, 49, 55, 48]",node time req_id od 0 ...,729,7.29,10.675125,2.135025,8.5401


# calculations 

In [43]:
inData.sblts.rides

,indexes,u_pax,u_veh,kind,u_paxes,times,indexes_orig,indexes_dest,degree,index,...,row,selected,nodes,req_id,sim_schedule,ttrav,dist,fare,commission,driver_revenue
0,[0],6.687500,361,1,[6.6875],"[0, 361]",[0],[0],1,0,...,"[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 44811672, 630748802]","[None, 0, 0]",node time req_id od 0 N...,361,3.61,5.424000,1.084800,4.3392
1,[1],2.876500,155,1,[2.8765],"[112, 155]",[1],[1],1,1,...,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44871798, 44884119]","[None, 1, 1]",node time req_id od 0 NaN...,155,1.55,2.334000,0.466800,1.8672
2,[2],7.346000,397,1,[7.346],"[122, 397]",[2],[2],1,2,...,"[0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1448535781, 44844068]","[None, 2, 2]",node time req_id od 0 ...,397,3.97,5.956500,1.191300,4.7652
3,[3],1.154500,62,1,[1.1545],"[206, 62]",[3],[3],1,3,...,"[0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44844874, 3510592637]","[None, 3, 3]",node time req_id od 0 ...,62,0.62,0.937500,0.187500,0.7500
4,[4],5.656000,305,1,[5.656],"[340, 305]",[4],[4],1,4,...,"[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 44867170, 1391531584]","[None, 4, 4]",node time req_id od 0 ...,305,3.05,4.588500,0.917700,3.6708
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
394,"[16, 14, 24]",23.472025,1129,30,"[10.844825, 6.5196000000000005, 6.107600000000...","[678.5, 334, 374, 92, 26, 303]","[16, 14, 24]","[14, 16, 24]",3,394,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1402613351, 44840852, 3581094285, 44795...","[None, 16, 14, 24, 14, 16, 24]",node time req_id od 0 ...,1129,11.29,15.279750,3.055950,12.2238
395,"[48, 46, 49]",17.805225,664,32,"[8.90535, 4.378725, 4.5211500000000004]","[2964.0, 15, 207, 75, 209, 158]","[48, 46, 49]","[46, 49, 48]",3,395,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1,"[None, 1410536956, 44833192, 44858073, 4487134...","[None, 48, 46, 49, 46, 49, 48]",node time req_id od 0 ...,664,6.64,12.085875,2.417175,9.6687
396,"[48, 46, 44]",17.678400,635,32,"[8.932649999999999, 4.560725, 4.185025]","[2964.0, 15, 142, 175, 78, 225]","[48, 46, 44]","[46, 44, 48]",3,396,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1410536956, 44833192, 44864086, 4487134...","[None, 48, 46, 44, 46, 44, 48]",node time req_id od 0 ...,635,6.35,11.727000,2.345400,9.3816
397,"[48, 49, 55]",17.133850,729,32,"[9.499125, 4.992075, 2.6426499999999997]","[2877.5, 222, 307, 22, 121, 57]","[48, 49, 55]","[49, 55, 48]",3,397,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0,"[None, 1410536956, 44858073, 4655063484, 23419...","[None, 48, 49, 55, 49, 55, 48]",node time req_id od 0 ...,729,7.29,10.675125,2.135025,8.5401


In [44]:
sim = simulate(params = params, inData = inData,  logger_level = logging.CRITICAL) # simulate

13-09-23 13:00:11-INFO-Matching 200 trips to 399 rides in order to min u_veh
13-09-23 13:00:11-INFO-Problem solution: Optimal. 
Total costs for single trips:         52,303 
reduced by matching to:               46,846
0.0005
13-09-23 13:00:16-CRITICAL-this is request 0 with [0, 257, 259] available rides.
13-09-23 13:00:16-CRITICAL-ride 0 available [1]
13-09-23 13:00:16-CRITICAL-ride 257 available [1, 0]
13-09-23 13:00:16-CRITICAL-ride 259 available [1, 0]
13-09-23 13:00:16-CRITICAL-this is request 0 with [0, 257, 259] still available rides.
13-09-23 13:00:16-CRITICAL-vehicle 1 has 3 choices
0.0005
13-09-23 13:00:16-CRITICAL-this is request 1 with [1] available rides.
13-09-23 13:00:16-CRITICAL-ride 1 available [1]
13-09-23 13:00:16-CRITICAL-this is request 1 with [1] still available rides.
13-09-23 13:00:16-CRITICAL-vehicle 8 has 1 choices
0.0005
13-09-23 13:00:16-CRITICAL-this is request 2 with [2, 332] available rides.
13-09-23 13:00:16-CRITICAL-ride 2 available [1]
13-09-23 13:00:1

In [45]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,335.0,20.0,30.0,0.0,297.0,73.0,0.0,10.0,0,0,0,0,0,335.0,370.0,55.0,False
1,15.0,155.0,20.0,30.0,0.0,85.0,0.0,112.0,10.0,0,0,0,0,0,155.0,85.0,55.0,False
2,15.0,423.0,20.0,30.0,0.0,55.0,239.0,122.0,10.0,0,0,0,0,0,423.0,294.0,55.0,False
3,15.0,62.0,20.0,30.0,0.0,13.0,0.0,206.0,10.0,0,0,0,0,0,62.0,13.0,55.0,False
4,15.0,305.0,20.0,30.0,0.0,128.0,0.0,340.0,10.0,0,0,0,0,0,305.0,128.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,15.0,154.0,20.0,30.0,0.0,163.0,0.0,14077.0,10.0,0,0,0,0,0,154.0,163.0,55.0,False
196,15.0,128.0,20.0,30.0,0.0,44.0,0.0,14115.0,10.0,0,0,0,0,0,128.0,44.0,55.0,False
197,0.0,38.0,20.0,30.0,0.0,295.0,107.0,14137.0,10.0,0,0,0,0,0,38.0,402.0,40.0,False


In [46]:
sim.res[0].pax_exp

,ACCEPTS_OFFER,ARRIVES_AT_DROPOFF,ARRIVES_AT_PICKUP,DEPARTS_FROM_PICKUP,LOSES_PATIENCE,MEETS_DRIVER_AT_PICKUP,RECEIVES_OFFER,REQUESTS_RIDE,SETS_OFF_FOR_DEST,STARTS_DAY,IS_REJECTED_BY_VEHICLE,REJECTS_OFFER,ARRIVES_AT_DEST,PREFERS_OTHER_SERVICE,TRAVEL,WAIT,OPERATIONS,LOST_PATIENCE
pax,,,,,,,,,,,,,,,,,,
0,15.0,335.0,20.0,30.0,0.0,297.0,73.0,0.0,10.0,0,0,0,0,0,335.0,370.0,55.0,False
1,15.0,155.0,20.0,30.0,0.0,85.0,0.0,112.0,10.0,0,0,0,0,0,155.0,85.0,55.0,False
2,15.0,423.0,20.0,30.0,0.0,55.0,239.0,122.0,10.0,0,0,0,0,0,423.0,294.0,55.0,False
3,15.0,62.0,20.0,30.0,0.0,13.0,0.0,206.0,10.0,0,0,0,0,0,62.0,13.0,55.0,False
4,15.0,305.0,20.0,30.0,0.0,128.0,0.0,340.0,10.0,0,0,0,0,0,305.0,128.0,55.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,15.0,154.0,20.0,30.0,0.0,163.0,0.0,14077.0,10.0,0,0,0,0,0,154.0,163.0,55.0,False
196,15.0,128.0,20.0,30.0,0.0,44.0,0.0,14115.0,10.0,0,0,0,0,0,128.0,44.0,55.0,False
197,0.0,38.0,20.0,30.0,0.0,295.0,107.0,14137.0,10.0,0,0,0,0,0,38.0,402.0,40.0,False


In [47]:
sim.res[0].pax_exp.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Plotting/Waits/PooledRides1.csv')

In [48]:
sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv')
sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv')

In [439]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set_style("whitegrid")

# Assuming sim, sim.res[0], and sim.res[0].veh_exp are properly defined

# Add 'Vehicles' as a column
sim.res[0].pax_exp['wait'] = sim.res[0].veh_exp.index

# Set up the figure
plt.figure(figsize=(10, 6))  # Adjusted figure size

# Create the bar plot with custom bar colors
ax = sns.barplot(data=sim.res[0].pax_exp, x="Vehicles", y="REVENUE", color="skyblue")

# Add labels and title
plt.xlabel("Number of Vehicles")
plt.ylabel("Revenue")
plt.title("Revenue by Number of Vehicles")

# Rotate x-axis labels for better readability
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")

# Adjust spacing between x-axis ticks and labels
plt.tight_layout()

# Show the plot
plt.show()


ValueError: Length of values (50) does not match length of index (500)

In [ ]:
sim.res[0].veh_exp.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Plotting/PrivateRide1.csv')


# Service rate 

In [ ]:
Fulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts().loc[False]
unFulfilled_requests = sim.res[0].pax_exp.LOST_PATIENCE.value_counts().loc[True]
service_rate = Fulfilled_requests/(Fulfilled_requests+unFulfilled_requests)
print('service_rate = ', service_rate*100, '%')



In [ ]:
#sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv')
#sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv')

In [ ]:
#for i in range(1, 2):
    #params.kpi = i
    #sim = simulate(params=params, inData = inData,  kpi_veh=kpi_veh_pricing,kpi_pax=kpi_pax_pricing, logger_level=logging.WARNING)
   # sim = simulate(params = params, inData = inData, logger_level = logging.CRITICAL) # simulate
    #sim.res[0].veh_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/veh{}.csv'.format(i))
   # sim.res[0].pax_kpi.to_csv('D:/Development/GitHub-ProjectV2.0/MaaSSim/docs/tutorials/Results/Simulation/Driver/Test/pax{}.csv'.format(i))
   

In [ ]:
#sim.vehs[2].rdf 

In [ ]:
#pd.DataFrame(sim.vehs[2].myrides)

## Script 